# Embedding Benchmark

##### Note: please install the following packages, pip install flair, pip install allennlp

#### The following embeddings are available for benchmark in this script: (1) USE, (2) BERT, (3) ELMo, (4) XLNet, (5) RoBERTa, (6) XLM, (7) GPT-2, (8) Transformer-XL, (9) Flair

#### Reminder: -------------PLEASE RUN BERT-AS-A-SERVICE-------------

## 1 - Import Packages

In [ ]:
from tqdm import tqdm

import pandas as pd

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import time
import tensorflow_hub as hub
import numpy as np
import tensorflow as tf

import torch
from pytorch_transformers import *

from bert_serving.client import BertClient

from flair.embeddings import *

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Lambda, Input, Embedding
import keras.backend as K
from keras.callbacks import ModelCheckpoint, TensorBoard

## 2 - Load Data

In [ ]:
# LOAD DATA SET
# THE FOLLOWING CODE TAKES IN A CSV WITH COLUMN 0 AS TEXT AND COLUMN 1 AS INTENT
data = pd.read_csv('path to csv file')
intent = list(data["intent"])
text = list(data["text"])

# TRAIN-TEST SPLIT
x_train, x_test, y_train, y_test = train_test_split(text, intent, test_size=0.2, random_state=0, stratify=intent)
train_dict = {'text':x_train, 'intent':y_train}
test_dict = {'text':x_test, 'intent':y_test}
train = pd.DataFrame.from_dict(train_dict)
test = pd.DataFrame.from_dict(test_dict)

# DISPLAY DATA SET DISTRIBUTION
print("Total Samples: \t\t\t", len(train)+len(test))
print("Samples in training set: \t", len(train))
print("Samples in testing set: \t", len(test))
print("\nNumber of intents: \t\t", len(data.iloc[:,1].unique()))
    
print("\nTraining distribution\n------------------------")
print(train.iloc[:, 1].value_counts())
print("\nTesting distribution\n-------------------------")
print(test.iloc[:, 1].value_counts())

## 3 - Embeddings

### 3.1 - USE

In [ ]:
class Universal():
    
    def __init__(self):
        self.use = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-large/3")
        self.n_labels = None
    
    def one_hot(self, label):
        le = LabelEncoder()
        le.fit(label)
        le_labels = le.transform(label)
        ohe_labels = to_categorical(le_labels)
        return ohe_labels
        
    def load_data(self, train, test):
        self.train_x = train.iloc[:,0]
        self.test_x = test.iloc[:,0]
        self.train_y = self.one_hot(list(train.iloc[:,1]))
        self.test_y = self.one_hot(list(test.iloc[:,1]))
        self.n_labels = len(train.iloc[:,1].unique())

    def embed(self, x):
        return self.use(tf.squeeze(tf.cast(x, tf.string)))
    
    def model(self):
        input_text = Input(shape=(1,), dtype='string')
        embedding = Lambda(self.embed, output_shape=(512,))(input_text)
        dense = Dense(256, activation='relu')(embedding)
        dense = Dropout(0.3)(dense)
        pred = Dense(self.n_labels, activation='softmax')(dense)
        self.model = Model(inputs=[input_text], outputs=pred)
        self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
    def train(self):
        with tf.Session() as session:
            K.set_session(session)
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer())
            ckpt = ModelCheckpoint('./model/use/{epoch:02d}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
            history = self.model.fit(np.asarray(self.train_x), np.asarray(self.train_y), validation_split=0.1, epochs=20, batch_size=64)
            self.model.save_weights('./model/use_model.h5')
            
    def evaluate(self):
        print("\nEvaluating......")
        with tf.Session() as session:
            K.set_session(session)
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer())
            self.model.load_weights('./model/use_model.h5')
            loss, acc = self.model.evaluate(np.asarray(self.test_x), np.asarray(self.test_y))
            print("\nThe loss is: ", loss, "The accuracy is: ", acc)

In [ ]:
use = Universal()
use.load_data(train, test)
use.model()
use.train()
use.evaluate()

### 3.2 - BERT

In [ ]:
class Bert():
    
    def __init__(self):
        self.bert = BertClient()
        self.n_labels = None
    
    def one_hot(self, label):
        le = LabelEncoder()
        le.fit(label)
        le_labels = le.transform(label)
        ohe_labels = to_categorical(le_labels)
        return ohe_labels
    
    def load_data(self, train, test):
        self.n_labels = len(train.iloc[:,1].unique())
        
        start = time.time()
        self.train_x = self.bert.encode(list(train.iloc[:,0]))
        end = time.time()
        print("Time taken to encode train: ", end-start)
        self.train_y = self.one_hot(train.iloc[:,1])
        start = time.time()
        self.test_x = self.bert.encode(list(test.iloc[:,0]))
        end = time.time()
        print("Time taken to encode test: ", end-start)
        self.test_y = self.one_hot(test.iloc[:,1])

        
    def model(self):
        self.model = Sequential()
        self.model.add(Dense(256, input_dim=1024, activation='relu'))
        self.model.add(Dropout(0.3))
        self.model.add(Dense(self.n_labels, activation='softmax'))
        self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        #model.summary()
        
    def train(self):
        with tf.Session() as session:
            K.set_session(session)
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer())
            ckpt = ModelCheckpoint('./model/bert/{epoch:02d}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
            history = self.model.fit(np.asarray(self.train_x), np.asarray(self.train_y), validation_split=0.1, epochs=20, batch_size=64, callbacks=[ckpt])
            self.model.save_weights('./model/bert_model.h5')
            
    def evaluate(self):
        with tf.Session() as session:
            K.set_session(session)
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer())
            self.model.load_weights('./model/bert_model.h5')
            loss, acc = self.model.evaluate(np.asarray(self.test_x), np.asarray(self.test_y))
            print("\nThe loss is: ", loss, "The accuracy is: ", acc)

In [ ]:
bert = Bert()
bert.load_data(train, test)
bert.model()
bert.train()
bert.evaluate()

### 3.3 - ELMo

In [ ]:
class ELMo():
    
    def __init__(self):
        self.elmo_embedding = ELMoEmbeddings('large')
        self.document_embeddings = DocumentPoolEmbeddings([self.elmo_embedding])
        self.n_labels = None
    
    def one_hot(self, label):
        le = LabelEncoder()
        le.fit(label)
        le_labels = le.transform(label)
        ohe_labels = to_categorical(le_labels)
        return ohe_labels
        
    def load_data(self, train, test):
        self.n_labels = len(train.iloc[:,1].unique())
        
        self.train_x = []
        self.test_x = []
        
        time.sleep(1)
        for t in tqdm(train.iloc[:,0].to_list(), desc="Train data: "):
            sentence = Sentence(t)
            self.document_embeddings.embed(sentence)
            self.train_x.append(sentence.get_embedding().detach().tolist())

        time.sleep(1)
        for t in tqdm(test.iloc[:,0].to_list(), desc="Test data: "):
            sentence = Sentence(t)
            self.document_embeddings.embed(sentence)
            self.test_x.append(sentence.get_embedding().detach().tolist())
        
        self.train_y = self.one_hot(list(train.iloc[:,1]))
        self.test_y = self.one_hot(list(test.iloc[:,1]))

        
    def model(self):
        self.model = Sequential()
        self.model.add(Dense(256, input_dim=3072, activation='relu'))
        self.model.add(Dropout(0.3))
        self.model.add(Dense(self.n_labels, activation='softmax'))
        self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        #model.summary()
        
    def train(self):
        with tf.Session() as session:
            K.set_session(session)
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer())
            ckpt = ModelCheckpoint('./model/elmo/{epoch:02d}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
            history = self.model.fit(np.asarray(self.train_x), np.asarray(self.train_y), validation_split=0.1, epochs=20, batch_size=64, callbacks=[ckpt])
            self.model.save_weights('./model/elmo_model.h5')
            
    def evaluate(self):
        with tf.Session() as session:
            K.set_session(session)
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer())
            self.model.load_weights('./model/elmo_model.h5')
            loss, acc = self.model.evaluate(np.asarray(self.test_x), np.asarray(self.test_y))
            print("\nThe loss is: ", loss, "The accuracy is: ", acc)

In [ ]:
elmo = ELMo()
elmo.load_data(train, test)
elmo.model()
elmo.train()
elmo.evaluate()

### 3.4 - XLNet

In [ ]:
class XLNet():
    
    def __init__(self):
        self.xlnet_embedding = XLNetEmbeddings(pooling_operation="mean")
        self.document_embeddings = DocumentPoolEmbeddings([self.xlnet_embedding])
        self.n_labels = None
    
    def one_hot(self, label):
        le = LabelEncoder()
        le.fit(label)
        le_labels = le.transform(label)
        ohe_labels = to_categorical(le_labels)
        return ohe_labels
        
    def load_data(self, train, test):
        self.n_labels = len(train.iloc[:,1].unique())
        
        self.train_x = []
        self.test_x = []
        
        time.sleep(1)
        for t in tqdm(train.iloc[:,0].to_list(), desc="Train data: "):
            sentence = Sentence(t)
            self.document_embeddings.embed(sentence)
            self.train_x.append(sentence.get_embedding().detach().tolist())

        time.sleep(1)
        for t in tqdm(test.iloc[:,0].to_list(), desc="Test data: "):
            sentence = Sentence(t)
            self.document_embeddings.embed(sentence)
            self.test_x.append(sentence.get_embedding().detach().tolist())
        
        self.train_y = self.one_hot(list(train.iloc[:,1]))
        self.test_y = self.one_hot(list(test.iloc[:,1]))

        
    def model(self):
        self.model = Sequential()
        self.model.add(Dense(256, input_dim=1024, activation='relu'))
        self.model.add(Dropout(0.3))
        self.model.add(Dense(self.n_labels, activation='softmax'))
        self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        #model.summary()
        
    def train(self):
        with tf.Session() as session:
            K.set_session(session)
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer())
            ckpt = ModelCheckpoint('./model/xlnet/{epoch:02d}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
            history = self.model.fit(np.asarray(self.train_x), np.asarray(self.train_y), validation_split=0.1, epochs=20, batch_size=64, callbacks=[ckpt])
            self.model.save_weights('./model/xlnet_model.h5')
            
    def evaluate(self):
        with tf.Session() as session:
            K.set_session(session)
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer())
            self.model.load_weights('./model/xlnet_model.h5')
            loss, acc = self.model.evaluate(np.asarray(self.test_x), np.asarray(self.test_y))
            print("\nThe loss is: ", loss, "The accuracy is: ", acc)

In [ ]:
XLnet = XLNet()
XLnet.load_data(train, test)
XLnet.model()
XLnet.train()
XLnet.evaluate()

### 3.5 - RoBERTa

In [ ]:
class Roberta():
    
    def __init__(self):
        self.roberta_embedding = RoBERTaEmbeddings('roberta-large', layers="-2", pooling_operation="mean")
        self.document_embeddings = DocumentPoolEmbeddings([self.roberta_embedding])
        self.n_labels = None
    
    def one_hot(self, label):
        le = LabelEncoder()
        le.fit(label)
        le_labels = le.transform(label)
        ohe_labels = to_categorical(le_labels)
        return ohe_labels
        
    def load_data(self, train, test):
        self.n_labels = len(train.iloc[:,1].unique())
        
        self.train_x = []
        self.test_x = []
        
        time.sleep(1)
        for t in tqdm(train.iloc[:,0].to_list(), desc="Train data: "):
            sentence = Sentence(t)
            self.document_embeddings.embed(sentence)
            self.train_x.append(sentence.get_embedding().detach().tolist())

        time.sleep(1)
        for t in tqdm(test.iloc[:,0].to_list(), desc="Test data: "):
            sentence = Sentence(t)
            self.document_embeddings.embed(sentence)
            self.test_x.append(sentence.get_embedding().detach().tolist())
        
        self.train_y = self.one_hot(list(train.iloc[:,1]))
        self.test_y = self.one_hot(list(test.iloc[:,1]))

        
    def model(self):
        self.model = Sequential()
        self.model.add(Dense(256, input_dim=1024, activation='relu'))
        self.model.add(Dropout(0.3))
        self.model.add(Dense(self.n_labels, activation='softmax'))
        self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        #model.summary()
        
    def train(self):
        with tf.Session() as session:
            K.set_session(session)
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer())
            ckpt = ModelCheckpoint('./model/roberta/{epoch:02d}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
            history = self.model.fit(np.asarray(self.train_x), np.asarray(self.train_y), validation_split=0.1, epochs=20, batch_size=64, callbacks=[ckpt])
            self.model.save_weights('./model/roberta_model.h5')
            
    def evaluate(self):
        with tf.Session() as session:
            K.set_session(session)
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer())
            self.model.load_weights('./model/roberta_model.h5')
            loss, acc = self.model.evaluate(np.asarray(self.test_x), np.asarray(self.test_y))
            print("\nThe loss is: ", loss, "The accuracy is: ", acc)

In [ ]:
roberta = Roberta()
roberta.load_data(train, test)
roberta.model()
roberta.train()
roberta.evaluate()

### 3.6 - XLM

In [ ]:
class XLM():
    
    def __init__(self):
        self.xlm_embedding = XLMEmbeddings(pooling_operation="mean")
        self.document_embeddings = DocumentPoolEmbeddings([self.xlm_embedding])
        self.n_labels = None
    
    def one_hot(self, label):
        le = LabelEncoder()
        le.fit(label)
        le_labels = le.transform(label)
        ohe_labels = to_categorical(le_labels)
        return ohe_labels
        
    def load_data(self, train, test):
        self.n_labels = len(train.iloc[:,1].unique())
        
        self.train_x = []
        self.test_x = []
        
        time.sleep(1)
        for t in tqdm(train.iloc[:,0].to_list(), desc="Train data: "):
            sentence = Sentence(t)
            self.document_embeddings.embed(sentence)
            self.train_x.append(sentence.get_embedding().detach().tolist())

        time.sleep(1)
        for t in tqdm(test.iloc[:,0].to_list(), desc="Test data: "):
            sentence = Sentence(t)
            self.document_embeddings.embed(sentence)
            self.test_x.append(sentence.get_embedding().detach().tolist())
        
        self.train_y = self.one_hot(list(train.iloc[:,1]))
        self.test_y = self.one_hot(list(test.iloc[:,1]))

        
    def model(self):
        self.model = Sequential()
        self.model.add(Dense(256, input_dim=2048, activation='relu'))
        self.model.add(Dropout(0.3))
        self.model.add(Dense(self.n_labels, activation='softmax'))
        self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        #model.summary()
        
    def train(self):
        with tf.Session() as session:
            K.set_session(session)
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer())
            ckpt = ModelCheckpoint('./model/xlm/{epoch:02d}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
            history = self.model.fit(np.asarray(self.train_x), np.asarray(self.train_y), validation_split=0.1, epochs=20, batch_size=64, callbacks=[ckpt])
            self.model.save_weights('./model/xlm_model.h5')
            
    def evaluate(self):
        with tf.Session() as session:
            K.set_session(session)
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer())
            self.model.load_weights('./model/xlm_model.h5')
            loss, acc = self.model.evaluate(np.asarray(self.test_x), np.asarray(self.test_y))
            print("\nThe loss is: ", loss, "The accuracy is: ", acc)

In [ ]:
XLM = XLM()
XLM.load_data(train, test)
XLM.model()
XLM.train()
XLM.evaluate()

### 3.7 - GPT-2

In [ ]:
class GPT2():
    
    def __init__(self):
        self.gpt2_embedding = OpenAIGPT2Embeddings()
        self.document_embeddings = DocumentPoolEmbeddings([self.gpt2_embedding])
        self.n_labels = None
    
    def one_hot(self, label):
        le = LabelEncoder()
        le.fit(label)
        le_labels = le.transform(label)
        ohe_labels = to_categorical(le_labels)
        return ohe_labels
        
    def load_data(self, train, test):
        self.n_labels = len(train.iloc[:,1].unique())
        
        self.train_x = []
        self.test_x = []
        
        time.sleep(1)
        for t in tqdm(train.iloc[:,0].to_list(), desc="Train data: "):
            sentence = Sentence(t)
            self.document_embeddings.embed(sentence)
            self.train_x.append(sentence.get_embedding().detach().tolist())

        time.sleep(1)
        for t in tqdm(test.iloc[:,0].to_list(), desc="Test data: "):
            sentence = Sentence(t)
            self.document_embeddings.embed(sentence)
            self.test_x.append(sentence.get_embedding().detach().tolist())
        
        self.train_y = self.one_hot(list(train.iloc[:,1]))
        self.test_y = self.one_hot(list(test.iloc[:,1]))

        
    def model(self):
        self.model = Sequential()
        self.model.add(Dense(256, input_dim=2048, activation='relu'))
        self.model.add(Dropout(0.3))
        self.model.add(Dense(self.n_labels, activation='softmax'))
        self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        #model.summary()
        
    def train(self):
        with tf.Session() as session:
            K.set_session(session)
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer())
            ckpt = ModelCheckpoint('./model/gpt-2/{epoch:02d}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
            history = self.model.fit(np.asarray(self.train_x), np.asarray(self.train_y), validation_split=0.1, epochs=20, batch_size=64, callbacks=[ckpt])
            self.model.save_weights('./model/gpt2_model.h5')
            
    def evaluate(self):
        with tf.Session() as session:
            K.set_session(session)
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer())
            self.model.load_weights('./model/gpt2_model.h5')
            loss, acc = self.model.evaluate(np.asarray(self.test_x), np.asarray(self.test_y))
            print("\nThe loss is: ", loss, "The accuracy is: ", acc)

In [ ]:
GPT2 = GPT2()
GPT2.load_data(train, test)
GPT2.model()
GPT2.train()
GPT2.evaluate()

### 3.8 - Transformer-XL

In [ ]:
class TransXL():
    
    def __init__(self):
        self.transxl_embedding = TransformerXLEmbeddings()
        self.document_embeddings = DocumentPoolEmbeddings([self.transxl_embedding])
        self.n_labels = None
    
    def one_hot(self, label):
        le = LabelEncoder()
        le.fit(label)
        le_labels = le.transform(label)
        ohe_labels = to_categorical(le_labels)
        return ohe_labels
        
    def load_data(self, train, test):
        self.n_labels = len(train.iloc[:,1].unique())
        
        self.train_x = []
        self.test_x = []
        
        time.sleep(1)
        for t in tqdm(train.iloc[:,0].to_list(), desc="Train data: "):
            sentence = Sentence(t)
            self.document_embeddings.embed(sentence)
            self.train_x.append(sentence.get_embedding().detach().tolist())

        time.sleep(1)
        for t in tqdm(test.iloc[:,0].to_list(), desc="Test data: "):
            sentence = Sentence(t)
            self.document_embeddings.embed(sentence)
            self.test_x.append(sentence.get_embedding().detach().tolist())
        
        self.train_y = self.one_hot(list(train.iloc[:,1]))
        self.test_y = self.one_hot(list(test.iloc[:,1]))

        
    def model(self):
        self.model = Sequential()
        self.model.add(Dense(256, input_dim=3072, activation='relu'))
        self.model.add(Dropout(0.3))
        self.model.add(Dense(self.n_labels, activation='softmax'))
        self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        #model.summary()
        
    def train(self):
        with tf.Session() as session:
            K.set_session(session)
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer())
            ckpt = ModelCheckpoint('./model/transxl/{epoch:02d}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
            history = self.model.fit(np.asarray(self.train_x), np.asarray(self.train_y), validation_split=0.1, epochs=20, batch_size=64, callbacks=[ckpt])
            self.model.save_weights('./model/transxl_model.h5')
            
    def evaluate(self):
        with tf.Session() as session:
            K.set_session(session)
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer())
            self.model.load_weights('./model/transxl_model.h5')
            loss, acc = self.model.evaluate(np.asarray(self.test_x), np.asarray(self.test_y))
            print("\nThe loss is: ", loss, "The accuracy is: ", acc)

In [ ]:
TXL = TransXL()
TXL.load_data(train, test)
TXL.model()
TXL.train()
TXL.evaluate()

### 3.9 - Flair

In [ ]:
class Flair():
    
    def __init__(self):
        self.glove_embedding = WordEmbeddings('glove')
        self.flair_embedding_forward = FlairEmbeddings('mix-forward')
        self.flair_embedding_backward = FlairEmbeddings('mix-backward')
        self.document_embeddings = DocumentPoolEmbeddings([self.glove_embedding,
                                              self.flair_embedding_backward,
                                              self.flair_embedding_forward])
        self.n_labels = None
        
    def one_hot(self, label):
        le = LabelEncoder()
        le.fit(label)
        le_labels = le.transform(label)
        ohe_labels = to_categorical(le_labels)
        return ohe_labels
        
    def load_data(self, train, test):
        self.n_labels = len(train.iloc[:,1].unique())
        
        self.train_x = []
        self.test_x = []
        
        time.sleep(1)
        for t in tqdm(train.iloc[:,0].to_list(), desc="Train data: "):
            sentence = Sentence(t)
            self.document_embeddings.embed(sentence)
            self.train_x.append(sentence.get_embedding().detach().tolist())

        time.sleep(1)
        for t in tqdm(test.iloc[:,0].to_list(), desc="Test data: "):
            sentence = Sentence(t)
            self.document_embeddings.embed(sentence)
            self.test_x.append(sentence.get_embedding().detach().tolist())
        
        self.train_y = self.one_hot(list(train.iloc[:,1]))
        self.test_y = self.one_hot(list(test.iloc[:,1]))

        
    def model(self):
        self.model = Sequential()
        self.model.add(Dense(256, input_dim=4196, activation='relu'))
        self.model.add(Dropout(0.3))
        self.model.add(Dense(self.n_labels, activation='softmax'))
        self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        #model.summary()
        
    def train(self):
        with tf.Session() as session:
            K.set_session(session)
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer())
            ckpt = ModelCheckpoint('./model/flair/{epoch:02d}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
            history = self.model.fit(np.asarray(self.train_x), np.asarray(self.train_y), validation_split=0.1, epochs=20, batch_size=64, callbacks=[ckpt])
            self.model.save_weights('./model/flair_model.h5')
            
    def evaluate(self):
        with tf.Session() as session:
            K.set_session(session)
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer())
            self.model.load_weights('./model/flair_model.h5')
            loss, acc = self.model.evaluate(np.asarray(self.test_x), np.asarray(self.test_y))
            print("\nThe loss is: ", loss, "The accuracy is: ", acc)

In [ ]:
Flair = Flair()
Flair.load_data(train, test)
Flair.model()
Flair.train()
Flair.evaluate()